## Digit Recognition

In [41]:
import tensorflow as tf
from os import path, getcwd, chdir
path = f"{getcwd()}/../tmp2/mnist.npz"

In [42]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [43]:
# callback function to stop training

class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('acc')>0.998):
                print("\nReached 99.8% accuracy so cancelling training!")
                self.model.stop_training = True

def train_mnist_conv():

    callbacks = myCallback()    
    mnist = tf.keras.datasets.mnist
    (training_images, training_labels), (test_images, test_labels) = mnist.load_data(path=path)

    training_images = training_images.reshape(60000, 28, 28, 1)
    test_images = test_images.reshape(10000, 28, 28, 1)
    
    # Normalizing List
    training_images = training_images / 255.0
    test_images = test_images/255.0

    # Define model
    model = tf.keras.models.Sequential([
             tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
             tf.keras.layers.MaxPooling2D(2, 2),
             tf.keras.layers.Flatten(),
             tf.keras.layers.Dense(128, activation='relu'),
             tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    
    # Train Model
    results = model.fit(training_images, training_labels, epochs=10, callbacks=[callbacks])
    trainAccuracy = (results.history['acc'][-1])
    print("Training Accuracy: {}".format(trainAccuracy))
    
    # Evaluate the model on the Test data ( Unseen Data )
    loss,testAccuracy = model.evaluate(test_images, test_labels)
    print("Loss: {}, Test Accuracy: {}".format(loss,testAccuracy))
    print("Difference in accuracy for training and test data is: {0:.2f} %".format((trainAccuracy - testAccuracy)*100))

In [44]:
train_mnist_conv()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 5408)              0         
_________________________________________________________________
dense_26 (Dense)             (None, 128)               692352    
_________________________________________________________________
dense_27 (Dense)             (None, 10)                1290      
Total params: 693,962
Trainable params: 693,962
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
60000/60000 [==============================] - 14s 233us/sample - loss: 0.1440 - acc: 0.9566
Epoch 2/1